In [ ]:
print("Hello")

In [ ]:
API_KEY="API_KEY"

In [ ]:
!pip install ipywidgets

In [ ]:


import requests
import json
import ipywidgets as widgets
from IPython.display import display

# Configuration dictionary with URL and API key separated

url= 'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key=API_KEY'  


prompt = """
Strictly work as a bot for dell services that guides the user.And dont answer questions that are not related to dell laptop.
For user greetings : greet politely to user 
provide complete installation steps of dell laptop without the user asking again and again.
if he asks for vs code installation then give vs code related installation steps clearly without the user asking again and again.
give all the settings like sound,wifi,network as the settings present in the dell laptop for the user you know.
For all these services you should respond according to the requested services with one line response for each respective request.
"""



# Function to get response from the API
def get_gemini_response(question,chat_history):
    try:
        # Prepare the payload including system prompt and user query
        history_text = "\n".join([f"{role}: {text}" for role, text in chat_history])
        payload = json.dumps({
            "contents": [
                {
                    "parts": [
                        {
                            "text": f"{prompt}\nConversation History:\n{history_text}\nUser Query: {question}"
                        }
                    ]
                }
            ]
        })

        # Define the headers
        headers = {
              # Include the API key for authorization
            'Content-Type': 'application/json'
        }

        # Make the POST request to the API
        response = requests.request("POST", url, headers=headers, data=payload)

        # Handle the API response
        if response.status_code == 200:
            try:
                response_json = response.json()
                # Assuming the response contains 'text' in the 'content' field
                response_text = response_json.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', 'No response text found.')
                return response_text
            except json.JSONDecodeError:
                return "Error parsing JSON response."
        else:
            return f"Request failed with status code {response.status_code}. {response.text}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

if 'chat_history' not in globals():
    chat_history = []

# Create text input and button widgets
text_input = widgets.Text(
    description='Input:',
    placeholder='Type your question here',
    style={'description_width': 'initial'}
)
button = widgets.Button(description="Ask the question")

# Define callback function
def on_button_click(b):
    global chat_history
    user_input = text_input.value
    if user_input:
        if user_input.lower() == 'end':
            # Clear the chat history and notify the user
            chat_history.clear() 
            print("Conversation cleared. You can start a new conversation.")
            text_input.value = ''
            return

        print("User input:", user_input)  # Debug: Print user input
        response_text = get_gemini_response(user_input,chat_history)
        print("Bot response:", response_text)  # Debug: Print bot response

        # Add user query and response to chat history
        chat_history.append(("You", user_input))
        chat_history.append(("Bot", response_text))
        
        # Clear the text input
        text_input.value = ''
        
        # Display the most recent interaction
        if chat_history:
            last_interaction = chat_history[-2:]  # Get the last two items
            display_text = "\n".join(f"{role}: {text}" for role, text in last_interaction)
            print(display_text)  # Print only the most recent interaction


# Attach callback to button
button.on_click(on_button_click)

# Display widgets
display(text_input, button)
